In [104]:
# This script uses functional roi niftis (converted from.img files and scaled in FSL to match
# size of each dimension). These parcels are used as binary masks on a t-statistical map output
# from first-level analysis. 
# This yields a hypothesis space for each roi (same location for every 
# subject). These roi's are taken from the Kanwisher website with the exception of DMFPC 
# (from saxelab).

# The top 50 voxels are then selected from these hypothesis space based on highest t-statistic
# (no threshold, and no constraint on contiguity).

# written by rania e. (rezzo@bu.edu)

In [ ]:
#####PATHWAYS

roi_size = 50; # of voxels you want in ROI

#import modules
import os
import numpy as np
import matplotlib
import nibabel as nib
import nilearn
from nibabel.testing import data_path
from nilearn import plotting
#from nilearn.image import smooth_img
from numpy import ndarray
import matplotlib.pyplot as plt

## load sample first level analysis output (t-map)
T_image = os.path.join('/om/group/saxelab/OpenAutism/Analysis/sample_first_level_output/tstat1.nii.gz')
t_image = nib.load(T_image)
# hyp_space_data = t_image.get_data()
t_data = np.array(t_image.dataobj)

## load sample level analysis output (con map)
# (do this later)

## where ROIs are located
root = '/om/group/saxelab/OpenAutism/Analysis/ROIs/'

#DMPFC
DMPFC = os.path.join(data_path, root + 'DMPFC_xyz_FSL_space.nii.gz')
Dmpfc = nib.load(DMPFC)
dmpfc_data = np.array(Dmpfc.dataobj)
dmpfc = Dmpfc.get_data()

#left FFA
LFFA = os.path.join(data_path, root + 'lFFA_FSL_space.nii.gz')
lffa = nib.load(LFFA)
lffa_data = np.array(lffa.dataobj)
lffa = lffa.get_data()

#left FFA
RFFA = os.path.join(data_path, root + 'rFFA_FSL_space.nii.gz')
rffa = nib.load(RFFA)
rffa_data = np.array(rffa.dataobj)

#left STS
LSTS = os.path.join(data_path, root + 'lSTS_FSL_space.nii.gz')
lsts = nib.load(LSTS)
lsts_data = np.array(lsts.dataobj)

#right STS
RSTS = os.path.join(data_path, root + 'rSTS_FSL_space.nii.gz')
rsts = nib.load(RSTS)
rsts_data = np.array(rsts.dataobj)

In [225]:
# multiply the t-image by the hypothesis space

# make all values in binary mask = 0 to NAN
# do this before: otherwise negative zeros can effect later calculations.
dmpfc_data[dmpfc_data == 0] = 'nan'

#multiply t-image by dmfpc roi mask;
dmpfc_hs = t_data*dmpfc_data

#convert to float
dmpfc_hs = dmpfc_hs.astype('float')


-1.91675
nan
nan
nan
7.39536571503


In [278]:
#### find top N t-values within this space to define ROI

#print highest values in the hypothesis space (t-values of the ROI)
# ! dont forget to later add error msg to hyp spaces that are < n
[values, indices] = nan_largestval(dmpfc_hs, roi_size)

#N highest t-values
print(values) 

#N indices of the highest t-values (each array x, arra y, array z)
print(indices) 

#sanity check
print(dmpfc_hs[43,85,53])

[ 7.39536572  6.48620081  6.48173809  6.34243298  6.29057932  5.78280926
  5.57839966  5.53596497  5.51747227  5.45706892  5.27595377  5.24354553
  5.22681904  5.171628    5.15757513  5.13953495  5.09517813  5.02283382
  4.99731302  4.99517918  4.96020031  4.93413067  4.89783669  4.79863405
  4.7853322   4.78332901  4.72948027  4.67820406  4.6113019   4.59164429
  4.56104708  4.5559783   4.48251152  4.45821857  4.37883902  4.35100603
  4.29260492  4.2741971   4.2685957   4.26255369  4.26122332  4.25771856
  4.20185089  4.18948317  4.10872793  4.0561552   4.0029211   3.98500657
  3.96539283  3.92076421]
(array([43, 37, 37, 37, 43, 43, 42, 35, 42, 43, 36, 37, 38, 39, 34, 40, 38,
       38, 42, 36, 43, 44, 43, 44, 36, 56, 44, 43, 56, 35, 37, 45, 41, 36,
       42, 42, 43, 45, 44, 36, 36, 35, 38, 41, 41, 37, 38, 42, 36, 36]), array([85, 94, 93, 93, 85, 85, 85, 91, 85, 84, 92, 91, 93, 92, 91, 92, 93,
       92, 85, 93, 87, 85, 86, 84, 91, 87, 84, 86, 90, 92, 93, 84, 85, 89,
       84, 86, 8

In [276]:
# this works, but must add error msg for rois smaller that N (specificied voxels)
# double check by plotting

def nan_largestval(ary, n):
    flat = ary.flatten()
    values = -np.sort(-flat)# order from greatest to least -- nans at the end
    idx = (-flat).argsort()[:n]
    idx2 = np.unravel_index(idx, ary.shape)
    return [values[0:n], idx2]

In [4]:
#plot (to test)

def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
         axes[i].imshow(slice.T, cmap="gray", origin="lower")

slice_0 = hyp_space_data[26, :, :]
slice_1 = hyp_space_data[:, 30, :]
slice_2 = hyp_space_data[:, :, 16]
show_slices([slice_0, slice_1, slice_2])

#nib.save(slice_0, os.path.join('build','test4d.nii.gz'))


In [5]:
show_slices(hyp_space_data)